# Analysis of Elevation Profiles of Hiking Trails

## Introduction 
This project focuses on the analysis of elevation profiles of hiking trails. The data sources used are hiking trails from [OpenStreetMap (OSM)](https://www.openstreetmap.org) and elevation data from the [DMR 5G](https://ags.cuzk.cz/arcgis/rest/services/3D/dmr5g/ImageServer) obtained from the [Czech Office for Surveying, Mapping and Cadastre (ČÚZK)](https://cuzk.cz). The project primarily focuses on hiking trails marked by the [Czech Tourist Club (KČT)](https://www.kct.cz).


## Import of libraries

#### Import standard libraries
[numpy](https://numpy.org/)

#### Import libraries for data and geospatial data processing
[requests](https://docs.python-requests.org/)  
[geopandas](https://geopandas.org/)  
[rasterio](https://rasterio.readthedocs.io/)  
[shapely](https://shapely.readthedocs.io/)  

#### Import libraries for visualization
[matplotlib](https://matplotlib.org/)  
[folium](https://python-visualization.github.io/folium/)  

#### Import libraries for working with OSM data
[osmapi](https://osmapi.readthedocs.io/)  
[overpy](https://python-overpy.readthedocs.io/)

In [1]:
# Import Standard Libraries
import numpy as np

# Import Libraries for Data and Geospatial Processing
import requests
import geopandas as gpd
import rasterio
from rasterio.plot import show
from shapely.geometry import mapping, Point, LineString, MultiLineString, Polygon

# Import Libraries for Visualization
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster

# Import Libraries for Working with OSM Data
import osmapi
import overpy

## Initialize Overpass API

To use Overpass API with the Overpy library, it is important to initialize the API object. This object is used to send queries and retrieve data.

In [2]:
# Initialize Overpass API
api = overpy.Overpass()

## Download of turist routes

#### Getting the Area ID by Place Name

This function retrieves the area ID for a given place name using the Overpass API. The area ID is further used to obtain tourist routes in the selected area.

In [3]:
# Function to get the area ID by place name
def get_area_id(area_name):
    """
    Get the area ID for a given place name using the Overpass API.

    Parameters:
    area_name (str): The name of the place to search for.

    Returns:
    int or None: The ID of the area if found, otherwise None.
    """
    # Query to get the area ID for the given place name
    area_query = f"""
    [out:json];
    area["name"="{area_name}"]["boundary"="administrative"];
    out body;
    """
    area_result = api.query(area_query)
    if len(area_result.areas) == 0:
        #print(f"No area found for {area_name}")
        return None
    return area_result.areas[0].id

#### Fetch Hiking Routes by KČT for the Specified Area

This function retrieves the hiking routes managed by the Czech Tourist Club (KČT) for a specified area using the Overpass API. The area is identified by its ID.

In [4]:
# Function to fetch hiking routes by KČT for the specified area
def fetch_hiking_routes(area_id):
    """
    Fetch the hiking routes by KČT for the specified area using the Overpass API.

    Parameters:
    area_id (int): The ID of the area to search within.

    Returns:
    overpy.Result: The result containing the IDs of the hiking routes.
    """
    # Query to get the IDs of hiking routes by KČT for the specified area
    relation_query = f"""
    [out:json];
    area({area_id});
    relation(area)["route"="hiking"]["operator"="cz:KČT"];
    out ids;
    """
    result = api.query(relation_query)
    return result

#### Download a Relation by Its ID

This function downloads a relation by its ID using the Overpass API.

In [5]:
# Function to download a relation by its ID
def download_relation(relation_id):
    """
    Download a relation by its ID using the Overpass API.

    Parameters:
    relation_id (int): The ID of the relation to download.

    Returns:
    dict or None: The relation data if found, otherwise None.
    """
    try:
        with osmapi.OsmApi() as osm_api:
            return osm_api.RelationGet(relation_id)
    except osmapi.errors.ElementDeletedApiError:
        print(f"Relation with ID {relation_id} not found.")
        return None

#### Download Hiking Routes

This function downloads hiking routes for a specified area. It prompts the user to enter the name of an area, retrieves the area ID, fetches the hiking routes by KČT for that area, and downloads the route details.

In [6]:
def download_hiking_routes():
    """
    Download hiking routes for a specified area.

    Prompts the user to enter the name of an area, retrieves the area ID,
    fetches the hiking routes by KČT for that area, and downloads the route details.

    Returns:
    list or None: A list of hiking routes if found, otherwise None.
    """
    area_name = input("Enter the name of the area: ")
    area_id = get_area_id(area_name)
    
    if area_id is None:
        print(f"{area_name} not found")
        return None
    
    result = fetch_hiking_routes(area_id)
    print(result)
    
    relations = []
    for relation in result.relations:
        rel = download_relation(relation.id)
        if rel:
            relations.append(rel)
    
    if relations:
        print(f"{len(relations)} hiking routes for {area_name}.")
        return relations
    else:
        print(f"No hiking routes were found in {area_name}.")
        return None

#### Main Execution Block

This block of code asks the user for the name of the village for which the tourist routes should be found.

In [7]:
if __name__ == "__main__": relations = download_hiking_routes()

Enter the name of the area:  Plzeň


4 hiking routes for Plzeň.


#### Function to Extract Descriptive Route Information from a Relation

This function extracts descriptive information about a hiking route from a given relation. It retrieves various tags and organizes them into a dictionary.

In [8]:
# Function to extract descriptive route information from a relation
def extract_route_info(relation):
    """
    Extract descriptive route information from a relation.

    Parameters:
    relation (dict): The relation data from which to extract information.

    Returns:
    dict: A dictionary containing descriptive information about the route.
    """
    tags = relation.get('tag', {})
    route_info = {
        'id': relation['id'],
        'name': tags.get('name', 'Name not provided'),
        'official_name': tags.get('official_name', 'Official name not provided'),
        'route_type': tags.get('route', 'Unknown route type'),
        'operator': tags.get('operator', 'Operator not provided'),
        'symbol': tags.get('osmc:symbol', 'Symbol not provided'),
        'ref': tags.get('ref', 'Reference not provided'),
        'complete': tags.get('complete', 'Not specified'),
        'educational': tags.get('educational', 'Not specified'),
        'network': tags.get('network', 'Unknown network'),
        'kct_yellow': tags.get('kct_yellow', 'Not specified'),
        'note': tags.get('note', 'No note provided'),
    }
    return route_info

#### Print Route Information

This block of code prints descriptive information about each hiking route. It iterates through the list of relations, extracts the route information, and prints it in a readable format.

In [9]:
# Print route info
for rel in relations:
    route_info = extract_route_info(rel)
    print("Descriptive information about the path:")
    for key, value in route_info.items():
        print(f"{key}: {value}")
    print("\n")

Descriptive information about the path:
id: 87590
name: Name not provided
official_name: Official name not provided
route_type: hiking
operator: cz:KČT
symbol: yellow:white:yellow_bar
ref: 6624
complete: yes
educational: Not specified
network: rwn
kct_yellow: major
note: No note provided


Descriptive information about the path:
id: 132429
name: Name not provided
official_name: Official name not provided
route_type: hiking
operator: cz:KČT
symbol: green:white:green_bar
ref: 3601
complete: yes
educational: Not specified
network: rwn
kct_yellow: Not specified
note: No note provided


Descriptive information about the path:
id: 6268435
name: Name not provided
official_name: Official name not provided
route_type: hiking
operator: cz:KČT
symbol: yellow:white:yellow_bar
ref: 6732
complete: yes
educational: Not specified
network: lwn
kct_yellow: major
note: část bývalé 6624 - začíná ve Valše (oddělení od zbytku zjištěno 2016-05);na sever od Valchy přeznačeno (2018-05)


Descriptive informatio

#### Function to Determine the Trail Color from the "osmc:symbol" Tag

This function determines the color of a hiking trail based on the "osmc:symbol" tag. It maps specific keywords in the symbol to corresponding colors.


In [10]:
# Function to determine the trail color from the "osmc:symbol" tag
def get_trail_color(symbol):
    """
    Determine the trail color from the "osmc:symbol" tag.

    Parameters:
    symbol (str): The "osmc:symbol" tag value.

    Returns:
    str: The color of the trail.
    """
    # Examples of mapping the symbol to a color (you can add more symbols as needed)
    if 'yellow' in symbol:
        return 'yellow'
    elif 'red' in symbol:
        return 'red'
    elif 'green' in symbol:
        return 'green'
    elif 'blue' in symbol:
        return 'blue'
    else:
        return 'gray'  # Default color if no match

#### Function to Create a Route Color Dictionary

This function creates a dictionary where the key is the hiking route ID and the value is its color. It processes a list of relations, extracts route information, determines the color based on the `osmc:symbol` tag, and stores the results in a dictionary.

In [11]:
def create_route_color_dict(relations):
    """
    Create a dictionary where the key is the hiking route ID and the value is its color.

    Parameters:
    relations (list): A list of relations representing hiking routes.

    Returns:
    dict: A dictionary mapping route IDs to their colors.
    """
    route_color_dict = {}
    
    for rel in relations:
        # Get route information
        route_info = extract_route_info(rel)
        symbol = route_info.get('symbol', '')  # Get the osmc:symbol tag
        color = get_trail_color(symbol)  # Determine the color based on the symbol
        
        # Store in the dictionary
        route_color_dict[route_info['id']] = color
    
    return route_color_dict

#### Check and Print Route Colors

This block of code checks if any hiking routes were found. If routes are found, it creates a dictionary mapping route IDs to their colors and prints the dictionary. If no routes are found, it prints a message indicating that no routes were found.

In [12]:
if relations:
    route_color_dict = create_route_color_dict(relations)
    print("Dictionary with route IDs and colors:")
    for route_id, color in route_color_dict.items():
        print(f"Route ID: {route_id}, Color: {color}")
else:
    print("No hiking routes were found.")

Dictionary with route IDs and colors:
Route ID: 87590, Color: yellow
Route ID: 132429, Color: green
Route ID: 6268435, Color: yellow
Route ID: 7945044, Color: green


#### Function to Convert a Relation to a GeoDataFrame

This function converts a relation to a GeoDataFrame.

In [13]:
def relation_to_geodataframe(relation):
    """
    Convert a relation to a GeoDataFrame.

    Parameters:
    relation (dict): The relation data containing members.

    Returns:
    GeoDataFrame: A GeoDataFrame containing the geometries of the relation members.
    """
    members = relation.get('member', [])
    data = []
    
    for member in members:
        member_type = member.get('type')
        ref = member.get('ref')
        role = member.get('role', '')
        
        if member_type == 'way':
            # Fetch geometry for "way" using the API
            with osmapi.OsmApi() as api:
                way = api.WayGet(ref)
                nodes = way['nd']
                coords = [(api.NodeGet(node_id)['lon'], api.NodeGet(node_id)['lat']) for node_id in nodes]
                geometry = LineString(coords)
                data.append({
                    'type': member_type,
                    'ref': ref,
                    'role': role,
                    'geometry': geometry
                })

    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(data, geometry='geometry')
    return gdf

#### Function to Convert Relations to a GeoDataFrame Dictionary

This function converts a list of relations into a dictionary where each key is a relation ID and each value is a GeoDataFrame. It processes each relation, extracts its ID, and stores a placeholder for the GeoDataFrame.

In [14]:
def relations_to_geodataframe_dict(relations):
    """
    Convert a list of relations to a dictionary of GeoDataFrames.

    Parameters:
    relations (list): A list of relation objects.

    Returns:
    dict: A dictionary where the keys are relation IDs and the values are GeoDataFrames.
    """
    geodataframe_dict = {}  # Initialize an empty dictionary

    # Iterate through each relation in the list of relations
    for rel in relations:
        # Get the relation ID (you can use any other attribute of the relation instead)
        rel_id = rel['id']
        
        # For now, store None until we have a function to convert to GeoDataFrame
        geodataframe_dict[rel_id] = relation_to_geodataframe(rel)

    return geodataframe_dict

#### Create a Dictionary of GeoDataFrames from Relations

This code snippet creates a dictionary where each key is a relation ID and each value is a GeoDataFrame. It uses the `relations_to_geodataframe_dict` function to process a list of relations, extract their IDs, and convert them into GeoDataFrames.

In [15]:
# Create a dictionary where keys are relation IDs and values are GeoDataFrames
gdf_dict = relations_to_geodataframe_dict(relations)

# Test print
print(gdf_dict)

{87590:    type         ref role                                           geometry
0   way   376865495       LINESTRING (13.42411 49.75902, 13.42410 49.758...
1   way    32089948       LINESTRING (13.42429 49.75923, 13.42430 49.759...
2   way    32089951       LINESTRING (13.42311 49.75909, 13.42340 49.759...
3   way    28983077       LINESTRING (13.42264 49.76033, 13.42286 49.759...
4   way    28983078       LINESTRING (13.42252 49.76046, 13.42264 49.76033)
..  ...         ...  ...                                                ...
81  way    27779759       LINESTRING (13.30084 49.77082, 13.30109 49.770...
82  way  1167787525       LINESTRING (13.30084 49.77082, 13.30043 49.770...
83  way  1167787524       LINESTRING (13.29981 49.77082, 13.29966 49.77082)
84  way    27779766       LINESTRING (13.29966 49.77082, 13.29964 49.770...
85  way   146781400       LINESTRING (13.29829 49.77061, 13.29796 49.77052)

[86 rows x 4 columns], 132429:     type         ref role                       

#### Combine Dictionaries: Route ID -> (GeoDataFrame, Color)

This code snippet combines dictionaries to create a new dictionary where each key is a route ID and each value is a tuple containing a GeoDataFrame and a color. It uses the `gdf_dict` for GeoDataFrames and `route_color_dict` for colors.


In [16]:
# Combine dictionaries: Route ID -> (GeoDataFrame, Color)
combined_dict = {}

for route_id, gdf in gdf_dict.items():
    color = route_color_dict.get(route_id, 'gray')  # Default color 'gray' if ID is missing
    combined_dict[route_id] = {'geodataframe': gdf, 'color': color}

# Test print
for route_id, data in combined_dict.items():
    print(f"Route ID: {route_id}, Color: {data['color']}")
    print(data['geodataframe'].head())  # Print first few rows of GeoDataFrame
    print("\n")

Route ID: 87590, Color: yellow
  type        ref role                                           geometry
0  way  376865495       LINESTRING (13.42411 49.75902, 13.42410 49.758...
1  way   32089948       LINESTRING (13.42429 49.75923, 13.42430 49.759...
2  way   32089951       LINESTRING (13.42311 49.75909, 13.42340 49.759...
3  way   28983077       LINESTRING (13.42264 49.76033, 13.42286 49.759...
4  way   28983078       LINESTRING (13.42252 49.76046, 13.42264 49.76033)


Route ID: 132429, Color: green
  type        ref role                                           geometry
0  way  437105531       LINESTRING (12.99258 50.05080, 12.99255 50.05091)
1  way  364615133       LINESTRING (12.99654 50.05037, 12.99596 50.050...
2  way  364615126       LINESTRING (13.00940 50.05012, 13.00930 50.050...
3  way  364615125       LINESTRING (13.01505 50.05073, 13.01455 50.050...
4  way  364615122       LINESTRING (13.02277 50.04773, 13.02242 50.048...


Route ID: 6268435, Color: yellow
  type       

#### Create a Basic Map Centered on the Czech Republic

This code snippet creates a basic map using Folium, adds a `MarkerCluster` for better visualization, and iterates through routes to add them to the map. It calculates a point on the line for placing text labels, ensuring the labels are on the line.

In [17]:
# Vytvoření základní mapy
m = folium.Map(location=[49.5, 13.5], zoom_start=7)

# Přidání MarkerCluster pro lepší zobrazení
marker_cluster = MarkerCluster().add_to(m)

# Iterace přes trasy a přidání do mapy
for route_id, data in combined_dict.items():
    # Centroid trasy pro umístění textového popisku
    centroid = data['geodataframe'].geometry.centroid.unary_union
    
    if centroid:
        # Výpočet průměrného bodu, pokud je centroid typu MultiPoint
        if centroid.geom_type == 'MultiPoint':
            centroid_x = sum([point.x for point in centroid.geoms]) / len(centroid.geoms)
            centroid_y = sum([point.y for point in centroid.geoms]) / len(centroid.geoms)
        else:
            centroid_x, centroid_y = centroid.x, centroid.y

        # Přidání trasy jako GeoJSON
        folium.GeoJson(
            mapping(data['geodataframe'].geometry.unary_union),
            style_function=lambda feature, color=data['color']: {
                'fillColor': color, 'color': color, 'weight': 2, 'fillOpacity': 0.5
            }
        ).add_to(marker_cluster)

        # Přidání popisku s ID trasy
        folium.Marker(
            location=[centroid_y, centroid_x],
            popup=f"ID: {route_id}",
            icon=folium.DivIcon(
                icon_size=(150, 36), icon_anchor=(7, 20),
                html=f'<div style="font-size: 10pt; color: {data["color"]};">{route_id}</div>'
            )
        ).add_to(marker_cluster)

m

#### Select a GeoDataFrame by Route ID

This function allows the user to select a GeoDataFrame from a dictionary by entering a route ID. It verifies the input and checks if the route ID exists in the dictionary.

In [18]:
def select_geodataframe(gdf_dict, route_id):
    # Verify that the route ID exists in the dictionary
    if route_id in gdf_dict:
        geodataframe = gdf_dict[route_id]  # Return the GeoDataFrame for the given ID
        
        # Check if the GeoDataFrame is empty
        if geodataframe.empty:
            print(f"The GeoDataFrame for route {route_id} is empty.")
            return None
        
        print(f"GeoDataFrame for route ID {route_id} has been selected.")
        return geodataframe
    else:
        print(f"Route ID {route_id} is not in the dictionary.")
        return None

## Call the Function and Work with the Selected GeoDataFrame

This code snippet calls the `select_geodataframe` function to select a GeoDataFrame from the dictionary and then works with the selected GeoDataFrame if it is not `None`.

In [33]:
# Example usage
route_id = 87590  # Replace with the desired route ID
gdf = select_geodataframe(gdf_dict, route_id)

GeoDataFrame for route ID 87590 has been selected.


In [34]:
from geopandas import GeoDataFrame

if isinstance(selected_gdf, GeoDataFrame):
    print("selected_gdf je GeoDataFrame.")
else:
    print("selected_gdf není GeoDataFrame.")

selected_gdf je GeoDataFrame.


In [35]:
# Getting info about the route

trail_symbol = route_info.get('symbol')

trail_color = get_trail_color(trail_symbol)

route_name = route_info.get('name', 'No name provided')

In [39]:
bounds = gdf.total_bounds

center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]

m = folium.Map(location=center, zoom_start=14)

gdf.explore(m=m, tiles="carto positron", color=trail_color)

display(m)

In [41]:
import geopandas as gpd

def get_bounding_box(gdf):
    # Getting the bounds of the GeoDataFrame (bounding box)
    bbox = gdf.total_bounds  # Returns [minx, miny, maxx, maxy]
    return bbox

# Příklad použití:
# Předpokládáme, že máte GeoDataFrame 'gdf'
# bbox = get_bounding_box(gdf)

In [48]:
import sys
import locale

# Získání výchozího kódování
default_encoding = sys.getdefaultencoding()

# Získání preferovaného kódování
preferred_encoding = locale.getpreferredencoding()

print(f"Default Encoding: {default_encoding}")
print(f"Preferred Encoding: {preferred_encoding}")

Default Encoding: utf-8
Preferred Encoding: UTF-8
